## Is there a relation between nucleic acid conformation and stacking with the protein?
- Which features of the nucleic acid conformation?
- How do we define stacking?

In [1]:
import json
from seamless.highlevel import Context, Cell, Transformer, Module

ctx = Context()
ctx.add_zip("initial-port.zip")
ctx.set_graph(json.load(open("initial-port.seamless")), mounts=True)
await ctx.translation()

In [2]:
# HACK: X3DNA computation is non-reproducible. Never forget a result!
from seamless.core.cache.buffer_cache import buffer_cache
buffer_cache.LIFETIME_TEMP = 999999
buffer_cache.LIFETIME_TEMP_SMALL = 999999

**Select PDB**

In [3]:
ctx.pdb_code = "1B7F"
ctx.na_chain = "P"
ctx.protein_chain = "A"
ctx.na_resid = 5
ctx.protein_resid = 256

**Visualize PDB**

In [4]:
import nglview
widget = nglview.NGLWidget()

ngl_args = {}
def show_ngl(*args, **kwargs):
    if len(args):
        variable = args[0]["owner"].path[-1]
        value = args[0]["new"]
        ngl_args[variable] = value
    if len(ngl_args) < 5:
        return
    pdb_code = ngl_args["pdb_code"]
    na_chain = ngl_args["na_chain"]
    protein_chain = ngl_args["protein_chain"]
    na_resid = ngl_args["na_resid"]
    protein_resid = ngl_args["protein_resid"]

    widget.clear()
    widget.add_component("rcsb://" + pdb_code)
    selection='({0} and :{1}) or ({2} and :{3})'.format(na_resid, na_chain, protein_resid, protein_chain)
    widget.add_representation('ball+stick', selection=selection, color='blue')
    widget.center(selection)

display(widget)

ctx.pdb_code.traitlet().observe(show_ngl)
ctx.na_chain.traitlet().observe(show_ngl)
ctx.na_resid.traitlet().observe(show_ngl)
ctx.protein_chain.traitlet().observe(show_ngl)
ctx.protein_resid.traitlet().observe(show_ngl)

NGLWidget()

In [5]:
ctx.na_resid = 6

In [6]:
ctx.na_resid = 5

In [7]:
from ipywidgets import IntText, Text

w_pdb_code= Text(
    description="Nucleic acid - protein complex PDB code", 
    style = {'description_width': 'initial'}
)
w_na_chain= Text(
    description="Nucleic acid chain ID", 
    style = {'description_width': 'initial'}
)
w_na_resid = IntText(
    description="Nucleic acid residue ID", 
    style = {'description_width': 'initial'}
)
w_protein_chain = Text(
    description="Protein chain ID", 
    style = {'description_width': 'initial'}
)
w_protein_resid = IntText(
    description="Protein residue ID", 
    style = {'description_width': 'initial'}
)
ctx.pdb_code.traitlet().link(w_pdb_code)
ctx.na_chain.traitlet().link(w_na_chain)
ctx.na_resid.traitlet().link(w_na_resid)
ctx.protein_resid.traitlet().link(w_protein_resid)
ctx.protein_chain.traitlet().link(w_protein_chain)
display(w_pdb_code)
display(w_na_chain)
display(w_na_resid)
display(w_protein_chain)
display(w_protein_resid)

Text(value='', description='Nucleic acid - protein complex PDB code', style=DescriptionStyle(description_width…

Text(value='', description='Nucleic acid chain ID', style=DescriptionStyle(description_width='initial'))

IntText(value=0, description='Nucleic acid residue ID', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Protein chain ID', style=DescriptionStyle(description_width='initial'))

IntText(value=0, description='Protein residue ID', style=DescriptionStyle(description_width='initial'))

**Execute X3DNA-DSSR**

In [8]:
ctx.x3dna_analysis.output()

Output()

**Select chain and show X3DNA results**

In [9]:
ctx.df_x3dna.output()

Output()

**Parse PDB into structured Numpy array using parse_pdb.py**

In [10]:
ctx.df_pdb.output()

Output()

**Integrate all properties for the selected interaction**
- Stacking properties are computed using the code above
- A list of other properties is extracted from the X3DNA analysis

In [11]:
ctx.all_properties.output()

Output()

**Calculate properties for all residue-nucleotide pair**

Instead of using the pre-selected residue and nucleotide, iterate over all

In [12]:
ctx.df_stackings.output()

Output()

**Plot a nucleotide conformation property versus a stacking property**

In [13]:
ctx.plot.output()

Output()

**Mount property calculation code and plotting code to file**

In [14]:
ctx.calc_properties.mount("calc-properties.py")
ctx.get_plot.code.mount("get-plot.py")
await ctx.translation()

**Share the plot over HTTP. It will be available as http://localhost:5813/ctx/plot.html**

In [15]:
ctx.plot.share("plot.html")
await ctx.translation()

Opened the seamless share update server at port 5138
Opened the seamless REST server at port 5813


**Alternative PDB structure to analyze**

Enter the following in the widgets or in a cell

```python
ctx.pdb_code = "1A9N"
ctx.na_chain = "Q"
ctx.protein_chain = "B"
ctx.na_resid = 13
ctx.protein_resid = 56
```